## 8.5 生成式对抗网络简介
### 8.5.3 生成器
**代码清单 8-29** GAN生成器网络

In [ ]:
import keras
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

# 将输入转换为大小为16x16的128个通道的特征图
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

# 添加卷积层
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# 上采样为 32x32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

# 更多的卷积层
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# 生成 32x32 单通道特征图（即CIFAR10图像形状）
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
# 将生成器模型实例化，它将形状为(latent_dim,)的输入映射到形状为(32, 32, 3)的图像
generator = keras.models.Model(generator_input, x)
generator.summary()

### 8.5.4 判别器
**代码清单 8-30** GAN判别器网络

In [ ]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

# 一个dropout层：重要技巧！
x = layers.Dropout(0.4)(x)

# 分类层
x = layers.Dense(1, activation='sigmoid')(x)

# 将判别器模型实例化，它将形状为(32, 32, 3)的输入转换为一个
# 二进制分类决策（真/假）
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

# 为了稳定训练过程，使用学习率衰减
# 以及在优化器中使用梯度裁剪（限制梯度值的范围）
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

### 8.5.5 对抗网络
**代码清单 8-31** 对抗网络

In [ ]:
# 将判别器权重设置为不可训练
# (仅应用于gan模型)
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

### 8.5.6 如何训练DCGAN
**代码清单 8-32** 实现GAN的训练

In [ ]:
import os
from keras.preprocessing import image

# 加载 CIFAR10 数据
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

# 选择青蛙图像 (class 6)
x_train = x_train[y_train.flatten() == 6]

# 数据标准化
x_train = x_train.reshape(
    (x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = './gan_images/'

# 开始循环
start = 0
for step in range(iterations):
    # 在潜在空间中采样随机点
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 将这些点解码为伪图像
    generated_images = generator.predict(random_latent_vectors)

    # 与真实图像混合
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # 合并标签，区分真实和虚假点图像
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # 向标签中添加噪声，重要技巧！
    labels += 0.05 * np.random.random(labels.shape)

    # 训练判别器
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # 在潜在空间中采样随机点
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 合并标签，全部是“真实标签”（谎言）
    misleading_targets = np.zeros((batch_size, 1))

    # 通过gan模型训练生成器,
    # 此时冻结判别器权重
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    # 每100步保存并绘图
    if step % 100 == 0:
        # 保存模型权重
        gan.save_weights('gan.h5')

        # 打印指标
        print('discriminator loss at step %s: %s' % (step, d_loss))
        print('adversarial loss at step %s: %s' % (step, a_loss))

        # 保存一张生成点图像
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))

        # 保存一张真实图像，用于对比
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))